In [73]:
# Install necessary libraries. 
!pip install pandas
!pip install statsmodels

In [74]:
# import necessary libraries. 
import pandas as pd
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

In [75]:
# Read in the data sets. 
df_may = pd.read_csv('data/filtered_stocks_11May.csv')
df_stock_outstanding = pd.read_csv('data/stock_descriptions_with_marketCap.csv')

In [76]:
# Separating SPY and other stocks. 
stocks_to_remove = ['DIA', 'ONEQ', 'SPY', 'VOO']
filtered_df_may = df_may[~df_may['Ticker_Symbol'].isin(stocks_to_remove)]

filtered_df_may_spy = df_may[df_may['Ticker_Symbol'] == 'SPY']

In [77]:
# rm is the daily market return.
rm_may = {'Date-Time': filtered_df_may_spy['Date-Time'], 'RM': (filtered_df_may_spy['Close'] - filtered_df_may_spy['Open']) / filtered_df_may_spy['Open'] * 100}
df_rm_may = pd.DataFrame(rm_may)

# rt is the daily return for each stock. 
rt_may = {'Date-Time': filtered_df_may['Date-Time'], 'RT': (filtered_df_may['Close'] - filtered_df_may['Open']) / filtered_df_may['Open'] * 100, 'Ticker_Symbol': filtered_df_may['Ticker_Symbol']}
df_rt_may = pd.DataFrame(rt_may)

# Extract different stock symbols.
unique_symbol_may = filtered_df_may['Ticker_Symbol'].unique()
grouped_df_may = filtered_df_may.groupby('Ticker_Symbol')

In [78]:
# Calculate the error term in the linear regression.
all_res = {}

for ticker in unique_symbol_may:
    each_ticker = df_rt_may[df_rt_may['Ticker_Symbol'] == ticker]
    aligned_data = pd.merge(df_rm_may, each_ticker, on='Date-Time', suffixes=('_SPY', '_Ticker'))
    x = aligned_data['RM'].values.reshape(-1, 1)
    y = aligned_data['RT'].values.reshape(-1, 1)
    
    model = LinearRegression()
    model.fit(x, y)

    y_pred = model.predict(x) 

    daily_error = y - y_pred
    
    all_res[ticker] = list(zip(aligned_data['Date-Time'], daily_error.flatten()))

for ticker, records in all_res.items():
    print(f"Details for {ticker}:")
    for date, error in records:
        print(f"Date: {date}, Error: {error:.4f}")
    print()

Details for HRL:
Date: 2023-06-12, Error: -0.2510
Date: 2023-06-09, Error: 0.7589
Date: 2023-06-08, Error: 0.6137
Date: 2023-06-07, Error: 1.1308
Date: 2023-06-06, Error: -0.6422
Date: 2023-06-05, Error: -0.4439
Date: 2023-06-02, Error: 1.0486
Date: 2023-06-01, Error: -0.0935
Date: 2023-05-31, Error: -0.6310
Date: 2023-05-30, Error: 0.5600
Date: 2023-05-26, Error: 0.5257
Date: 2023-05-25, Error: -0.9375
Date: 2023-05-24, Error: -0.5052
Date: 2023-05-23, Error: 0.3758
Date: 2023-05-22, Error: -1.0134
Date: 2023-05-19, Error: 0.1470
Date: 2023-05-18, Error: 0.0702
Date: 2023-05-17, Error: 0.6837
Date: 2023-05-16, Error: -0.8862
Date: 2023-05-15, Error: -1.7270
Date: 2023-05-12, Error: 0.2430
Date: 2023-05-11, Error: 0.2514
Date: 2023-05-10, Error: 0.0874
Date: 2023-05-09, Error: -0.8097
Date: 2023-05-08, Error: -0.5307
Date: 2023-05-05, Error: 0.4242
Date: 2023-05-04, Error: 0.6349
Date: 2023-05-03, Error: 0.8446
Date: 2023-05-02, Error: -0.5728
Date: 2023-05-01, Error: 0.2347
Date: 2023

In [79]:
def calculate_CAR(errors_with_dates, ticker, event_date_str, k):
    """
    Calculate the Cumulative Abnormal Return (CAR) for a given ticker around an event date,
    considering only the dates present in the errors_with_dates dataset.

    Parameters:
    - errors_with_dates: Dict, with ticker symbols as keys and lists of (date, error) tuples as values.
    - ticker: String, the ticker symbol for which to calculate CAR.
    - event_date_str: String, the event date in 'YYYY-MM-DD' format.
    - k: Integer, the number of days before and after the event date to consider, based on available data.

    Returns:
    - CAR: Float, the Cumulative Abnormal Return for the specified window around the event date.
    """
    event_date = datetime.strptime(event_date_str, '%Y-%m-%d')
    date_errors = sorted(errors_with_dates.get(ticker, []), key=lambda x: datetime.strptime(x[0], '%Y-%m-%d'))
    event_date_indexes = [i for i, date_error in enumerate(date_errors) if date_error[0] == event_date_str]
    
    if not event_date_indexes:
        return 0 
    
    event_date_index = event_date_indexes[0]
    
    start_index = max(0, event_date_index - k)
    end_index = min(len(date_errors) - 1, event_date_index + k)
    CAR = sum(error for _, error in date_errors[start_index:end_index + 1])

    return CAR

In [80]:
# filtered_df_may is the df without 'DIA', 'ONEQ', 'SPY', 'VOO'
# volatility_may is the filtered_df_may with a 'Volatility' column 
volatility_may = filtered_df_may.copy()
volatility_may['Volatility'] = volatility_may['High'] - volatility_may['Low']

In [81]:
# merged_may only contains columns ['Date-Time', 'Open', 'Close', 'High', 'Low', 'Volume', 'Ticker_Symbol', 'Volatility', 'Sector', 'Industry', '2023']
temp_df_may = volatility_may.copy() # To store the original volatility_may data
merged_may = pd.merge(temp_df_may, df_stock_outstanding, left_on='Ticker_Symbol', right_on='Symbol', how='left')
merged_may = merged_may[['Date-Time', 'Open', 'Close', 'High', 'Low', 'Volume', 'Ticker_Symbol', 'Volatility', 'Sector', 'Industry', '2023']]

In [82]:
# Turnover of may (2023)
df_turnover_may = merged_may.copy()
df_turnover_may['Turnover'] = df_turnover_may['Volume'] / df_turnover_may['2023']

In [83]:
# Calculate Market Capitalization
df_market_cap_2023 = df_turnover_may.copy()
df_market_cap_2023['Market_Cap'] = np.log(df_market_cap_2023['Close'] * df_market_cap_2023['2023'])

In [84]:
# Change Inductry to numbers
# Finalised DataFrame df_2023
# 2023
df_2023 = df_market_cap_2023.copy()
df_2023['Industry_name'] = df_2023['Industry']
df_2023['Industry'] = df_2023['Industry'].astype('category')
df_2023['Industry'] = df_2023['Industry'].cat.codes

In [85]:

# event date 2023-05-11
temp_df_2023 = df_2023.copy()

features_list = [] 

for ticker in temp_df_2023['Ticker_Symbol'].unique():
    selected_rows = temp_df_2023[temp_df_2023['Ticker_Symbol'] == ticker]
    selected_row = selected_rows[selected_rows['Date-Time'] == '2023-05-11']
    
    y_value = calculate_CAR(all_res, ticker, '2023-05-11', 7)
    if not selected_row.empty: 
        X = selected_row[['Industry', 'Turnover', 'Market_Cap', 'Close', 'Volatility']]
        X = pd.get_dummies(X, columns=['Industry'], drop_first=True)
        features = X.iloc[0].values 
        features_list.append(np.append(features, y_value)) 

df_features = pd.DataFrame(features_list)
X = df_features.iloc[:, :-1]
y = df_features.iloc[:, -1]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      4   R-squared:                       0.312
Model:                            OLS   Adj. R-squared:                  0.174
Method:                 Least Squares   F-statistic:                     2.265
Date:                Sat, 23 Mar 2024   Prob (F-statistic):             0.0981
Time:                        23:09:55   Log-Likelihood:                -67.903
No. Observations:                  25   AIC:                             145.8
Df Residuals:                      20   BIC:                             151.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         40.8145     14.926      2.735      0.0